<a href="https://colab.research.google.com/github/dwivedys/Python_Programs/blob/master/GooglePayAnalysisFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
# This program prints out the total payments done using Google Pay
# I have downloaded my transactions from Google Pay by requesting Google
# Google provides an HTML file for the transactions and hence the need for beautifulsoup to scrape the html
# This is a very basic attempt to try to figure out money paid to various parties during different time frames
# V 1.0
# Author: Saurabh Dwivedy

# Future improvements are planned

import re
from bs4 import BeautifulSoup
import pandas as pd
#import urllib.request as urllib2

HtmlFile = open('My Activity.html', 'r', encoding='utf-8')
source_code = HtmlFile.read() 
soup = BeautifulSoup(source_code, 'html.parser')

# Key Step
# The amounts and dates appear in a specific div tag
divTags = soup.findAll('div', {'class':'content-cell mdl-cell mdl-cell--6-col mdl-typography--body-1'})

z = []

for n in divTags:
  z.append([list(n.children)[0], list(n.children)[2]])

df = pd.DataFrame(z, columns = ["Transactions", "Date"])

k = []
for z in df['Transactions']:
  # Pick the financial transactions based on keywords
  if z.split(" ")[0] in ['Received', 'Sent', 'Paid']:
    k.append(z)
  else:
    k.append('NAN')


transaction_amts = []
for t in k:
  if t == 'NAN': 
    transaction_amts.append(0.0)
  else:
    # Extract the Amount field, convert to float and remove any formatting commas
    transaction_amts.append(float(re.split(r'\s', t)[1][1:].replace(",", "")))

# Populate the dataframe with Transaction_Amounts
df['Trx. Amt.'] = transaction_amts
df['Payment Description'] = k

Year = []

for x in df['Date']:
  if x.split(",")[0].split(" ")[2] == '2019':
    Year.append('2019')
  elif x.split(",")[0].split(" ")[2] == '2020':
    Year.append('2020')
  elif x.split(",")[0].split(" ")[2] == '2021':
    Year.append('2021')
  else: 
    Year.append('0000')
 
df['Year'] = Year

Type = []

for x in df['Payment Description']:
  if x.split(" ")[0] == 'Paid': 
    Type.append('Paid')
  elif x.split(" ")[0] == 'Received': 
    Type.append('Received')
  elif x.split(" ")[0] == 'Sent': 
    Type.append('Sent')
  else:
    Type.append('NAN')

df['Type'] = Type


# Sample Data Analysis - By Year
# df[df.Type != 'NAN'].groupby('Year')['Trx. Amt.'].sum()

# Sample Data Analysis - By Year, By Transaction Type

df[df.Type != 'NAN'].groupby(['Year', 'Type'])['Trx. Amt.'].sum()

# Sample Data Analysis - By Transaction Type

#df[df.Type != 'NAN'].groupby('Type')['Trx. Amt.'].sum()

#print('Yay finished!')

Year  Type    
2019  Paid          1390.0
      Sent         10206.0
2020  Paid        154047.0
      Received      5879.0
      Sent        613839.0
2021  Paid        203517.7
      Received        22.0
      Sent         50084.0
Name: Trx. Amt., dtype: float64